In [ ]:
!pip install pandas
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
## Isi nama dan NIM seluruh anggota kelompok di sini
#   Nama                            NIM
#   Mohammad Andiez Satria Permana  1301218598
#   HNW Syahuda Nahatmasuni         1301218603
#   Wandi Yusuf Kurniawan           1301218601

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree

# Load Dataset
Inisialisasi variable, masukan NIM di sini untuk random state.

Gunakan NIM salah satu anggota kelompok.



In [ ]:
filepath = "./sample_data/seeds_dataset.csv"

# Isi nim anda di sini
nim = 1301218601

Load dataset, cek konten data

Target kelas: **Category** 

In [ ]:
df = pd.read_csv(filepath, sep="\t")
df.sample(10, random_state=nim)

,Area,Perimeter,Compactness,Kernel_len,Kernel_width,Assymetry,Groove_len,Category
8,16.63,15.46,0.8747,6.053,3.465,2.040,5.877,1
67,14.01,14.29,0.8625,5.609,3.158,2.217,5.132,1
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
199,12.76,13.38,0.8964,5.073,3.155,2.828,4.830,3
26,13.02,13.76,0.8641,5.395,3.026,3.373,4.825,1
19,12.72,13.57,0.8686,5.226,3.049,4.102,4.914,1
70,17.63,15.98,0.8673,6.191,3.561,4.076,6.060,2
51,15.78,14.91,0.8923,5.674,3.434,5.593,5.136,1
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
169,11.24,13.00,0.8359,5.090,2.715,3.521,5.088,3


Training/test split **60/40**

In [ ]:
# X: semua kolom, Y: kolom terakhir (category)
x = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.6, random_state=nim)

# We perform bootstrap on the train data
# Keep test data for final accuracy testing
len(x_train), len(x_test)

(126, 84)

In [ ]:
x_train.head()

,Area,Perimeter,Compactness,Kernel_len,Kernel_width,Assymetry,Groove_len
157,12.13,13.73,0.8081,5.394,2.745,4.825,5.220
118,18.89,16.23,0.9008,6.227,3.769,3.639,5.966
159,11.49,13.22,0.8263,5.304,2.695,5.388,5.310
112,19.13,16.31,0.9035,6.183,3.902,2.109,5.924
83,19.57,16.74,0.8779,6.384,3.772,1.472,6.273


# Random Forest
Build a Random Forest, consisting of 20 Decision Tree classifer, with the following steps:

1. Prepare Bootstrap data (choose randomly from the *training data*, use sampling with replacement)
2. Choose 2-4 random features/atributes for each tree
3. Build Decision Tree classifier using step (1) and (2)
4. Calculate OOB score within the tree only

**NOTE**
This whole exercise has been simplified.
- Tree can be created using a library, therefore you may not be able control the feature selection on node level
- Random features are only selected in the beginning of each tree.
- OOB score is only calculated within its own tree instead.
- Typically Random Forest don't use train/test split and can be validated using OOB error. We simplify this to make it easier to test.
- You are not allowed to use a Random Forest or Bagging library.


In [ ]:
np.random.seed(nim)
n_tree = 20

forest = []
bootstrap_columns = []
for i in range(n_tree):
    # 1. Bootstrapping
    # select random rows
    rows = np.random.choice(len(x_train), size=42, replace=False)
    x_bootstrap = x_train.iloc[rows]
    y_bootstrap = y_train.iloc[rows]

    # Out of Bag data
    non_selected_rows = np.delete(np.arange(0, len(x_train)), rows)
    x_oob = x_train.iloc[non_selected_rows]
    y_oob = y_train.iloc[non_selected_rows]

    # 2. Select random features (make sure to keep track the selected columns)
    # Check for any repeated combinations
    feature_taken = np.random.choice(len(x_train.columns), size=np.random.randint(2, 5), replace=False)
    cols = x_train.columns[feature_taken]
    bootstrap_columns.append(cols)

    # filter based on the selected columns
    x_bootstrap = x_bootstrap.iloc[:, feature_taken]
    x_oob = x_oob.iloc[:, feature_taken]

    # 3. Build decision tree
    # Let it grow, no pruning
    tree_model = tree.DecisionTreeClassifier(criterion="entropy", max_depth=len(feature_taken)+1) #diperbolehkan

    # Add it to the forest
    forest.append(tree_model)

    # 4. Calculate oob score only within its own tree
    tree_model.fit(x_bootstrap, y_bootstrap)
    oob_score = np.sum(tree_model.predict(x_oob) == y_oob)/len(x_oob)
    print(cols, oob_score)



Index(['Kernel_len', 'Perimeter', 'Groove_len'], dtype='object') 0.8690476190476191
Index(['Assymetry', 'Compactness'], dtype='object') 0.5357142857142857
Index(['Area', 'Compactness'], dtype='object') 0.8333333333333334
Index(['Kernel_width', 'Compactness', 'Area', 'Kernel_len'], dtype='object') 0.8333333333333334
Index(['Kernel_width', 'Compactness'], dtype='object') 0.8095238095238095
Index(['Kernel_width', 'Kernel_len', 'Compactness'], dtype='object') 0.7738095238095238
Index(['Kernel_width', 'Compactness', 'Area', 'Assymetry'], dtype='object') 0.8571428571428571
Index(['Groove_len', 'Area', 'Perimeter'], dtype='object') 0.8095238095238095
Index(['Kernel_width', 'Assymetry'], dtype='object') 0.8571428571428571
Index(['Kernel_len', 'Assymetry', 'Perimeter', 'Area'], dtype='object') 0.7619047619047619
Index(['Groove_len', 'Kernel_width', 'Area'], dtype='object') 0.8928571428571429
Index(['Groove_len', 'Assymetry'], dtype='object') 0.8333333333333334
Index(['Kernel_width', 'Assymetry'

# Prediction and Voting
Run prediction on x_test data for each tree

Also calculate accuracy per tree here

In [75]:
all_preds = []
for i, tree_model in enumerate(forest):
    # Each tree has different column requirements
    # Make sure you use the correct columns for each tree
    x_test_filtered = x_test[bootstrap_columns[i]]
    predictions = tree_model.predict(x_test_filtered)

    # Might as well calculate accuracy per tree here...
    acc = np.sum(predictions == y_test) / len(predictions)
    print(acc)

    # add it to all_preds for voting later
    all_preds.append(predictions)

all_preds = np.array(all_preds)
print(all_preds)
print(all_preds.shape)

0.8571428571428571
0.5833333333333334
0.8809523809523809
0.7857142857142857
0.7738095238095238
0.7142857142857143
0.8452380952380952
0.8095238095238095
0.7857142857142857
0.8214285714285714
0.8690476190476191
0.7738095238095238
0.8571428571428571
0.7738095238095238
0.8571428571428571
0.7261904761904762
0.8214285714285714
0.7857142857142857
0.7142857142857143
0.8809523809523809
[[2 1 1 ... 1 1 2]
 [1 3 1 ... 1 3 1]
 [2 1 1 ... 1 1 2]
 ...
 [2 1 2 ... 1 2 2]
 [2 3 1 ... 1 3 2]
 [2 1 1 ... 3 1 2]]
(20, 84)


### Perform voting on the predictions from the Random Forest

In [98]:
# Insert voting code here
import statistics as st

all_preds = all_preds.T
voted_predictions = []
for i in enumerate(all_preds):
    voted_predictions.append(st.mode(all_preds[i]))

# print(voted_predictions)

# Calculate accuracy of Test data on the Forest
acc = np.sum(voted_predictions == y_test) / len(voted_predictions)
print(acc)



0.6904761904761905


After you're done with all this, please check the excel file and put the answers for every column there.